In [1]:
import praw
import json
import os
import matplotlib.pyplot as plt
import networkx as nx
from anytree import Node, RenderTree

import warnings 
warnings.filterwarnings('ignore')

In [2]:
client_id = 'wQEuOcq3Oxz8EJaf1hzigg'
client_secret = 'Walqn1Xz7h20dbvppmTcv16Fxl7uhg'
user_agent = 'sury-umanch'

In [3]:
import praw

# Initialize Reddit client
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)


Version 7.7.1 of praw is outdated. Version 7.8.1 was released Friday October 25, 2024.


## Search Submissions by Keywords on Subreddits

In [5]:
import csv

subreddit_name = "ukpolitics"
query = "universal income OR minimum wage OR council tax OR income tax OR national insurance OR poverty"
limit = 100  # PRAW's search isn't truly unlimited

# Store submission info
submissions_data = []

# Search subreddit with query
for submission in reddit.subreddit(subreddit_name).search(query, limit=limit):
    submissions_data.append({
        "submission_id": submission.id,
        "title": submission.title,
        "score": submission.score,
        "url": submission.url,
        "created_utc": submission.created_utc,
        "author": str(submission.author)
    })

# Save to CSV
with open("data/submissions.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=submissions_data[0].keys())
    writer.writeheader()
    writer.writerows(submissions_data)


## Get All comments from submnission data

In [10]:
import time

comments_data = []

start = time.time()

for sub in submissions_data:
    submission = reddit.submission(id=sub["submission_id"])
    submission.comments.replace_more(limit=None)  # Load all comments

    for comment in submission.comments.list():
        comments_data.append({
            "submission_id": submission.id,
            "comment_id": comment.id,
            "body": comment.body,
            "author": str(comment.author),
            "score": comment.score,
            "created_utc": comment.created_utc,
            "parent_id": comment.parent_id
        })

    time.sleep(1)  # Be respectful of Reddit's rate limits

    end = time.time()

    if (end-start) > 60:
        break

# Save to CSV
with open("data/comments.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=comments_data[0].keys())
    writer.writeheader()
    writer.writerows(comments_data)
